# Execution options

One of the key design features of Pydra is the separation between the parameterisation of
the task to be executed, and the parameresiation of where and how the task should be
executed (e.g. on the cloud, on a HPC cluster, ...). This tutorial steps you through
some of the available options for executing a task.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nipype/pydra-tutorial/develop/notebooks/tutorial/advanced_execution.ipynb)

In [1]:
import nest_asyncio
nest_asyncio.apply()

## Workers

Pydra supports several workers with which to execute tasks

- `cf`
- `slurm`
- `sge`
- `dask` (experimental)
- `debug`

By default, the *cf* (*ConcurrentFutures*) worker is used, which
executes tasks across multiple processes. If you are using a high-performance cluster (HPC)
then the [SLURM](https://slurm.schedmd.com/documentation.html) and
[SGE](https://www.metagenomics.wiki/tools/hpc-sge) workers can be used to submit each
workflow node as separate jobs to the HPC scheduler. There is also an
experimental [Dask](https://www.dask.org/) worker.

When using a debugger in the development of a workflow or Python tasks, the
*debug* worker is recommended as it executes nodes "synchronously" (as opposed to
asynchronously), and can therefore break on uncaught exceptions.

To specify a worker, the abbreviation can be passed either as a string or using the
class itself. Additional parameters can be passed to the worker initialisation as keyword
arguments to the execution call. For example, if we wanted to run five tasks using the
ConcurentFutures worker but only use three CPUs, we can pass `n_procs=3` to the execution
call.

In [2]:
from pydra.design import python

@python.define
def TenToThePower(p: int) -> int:
    return 10 ** p

ten_to_the_power = TenToThePower().split(p=[1, 2, 3, 4, 5])

# Run the 5 tasks in parallel split across 3 processes
outputs = ten_to_the_power(worker="cf", n_procs=3)

p1, p2, p3, p4, p5 = outputs.out

print(f"10^5 = {p5}")

A newer version (0.25) of nipype/pydra is available. You are using 0.25.dev141+g03c7438b.d20250123


IndentationError: unexpected indent (<unknown>, line 1)

## Cache locations

When a task runs, a unique hash is generated by the combination of all the inputs to the
task and the operation to be performed. This hash is used to name the output directory for
the task within the specified cache directory. Therefore, if you use the same cache
directory between runs and in a subsequent run the same task is executed with the same
inputs then the location of its output directory will also be the same, and the outputs
generated by the previous run are reused.

For example, using the MrGrid example from the [Getting Started Tutorial](./1-getting-started.html)


In [ ]:
from pathlib import Path
import tempfile
from fileformats.medimage import Nifti1
from pydra.engine.submitter import Submitter
from pydra.tasks.mrtrix3.v3_0 import MrGrid
import nest_asyncio

# Allow running async code in Jupyter notebooks
nest_asyncio.apply()

# Make directory filled with nifti files
test_dir = Path(tempfile.mkdtemp())
nifti_dir = test_dir / "nifti"
nifti_dir.mkdir()
for i in range(10):
    Nifti1.sample(nifti_dir, seed=i)

VOXEL_SIZES = [0.5, 0.5, 0.5, 0.75, 0.75, 0.75, 1.0, 1.0, 1.0, 1.25]

mrgrid_varying_vox_sizes = MrGrid().split(
    ("input", "voxel"),
    input=nifti_dir.iterdir(),
    voxel=VOXEL_SIZES
)

submitter = Submitter(cache_dir=test_dir / "cache")

# Run the task to resample all NIfTI files with different voxel sizes
with submitter:
    result1 = submitter(mrgrid_varying_vox_sizes)

If we attempt to run the same task with the same parameterisation the cache directory
will point to the same location and the results will be reused

In [ ]:
mrgrid_varying_vox_sizes2 = MrGrid().split(
    ("input", "voxel"),
    input=nifti_dir.iterdir(),
    voxel=VOXEL_SIZES
)

submitter = Submitter(cache_dir=test_dir / "cache")

# Result from previous run is reused as the task and inputs are identical
with submitter:
    result2 = submitter(mrgrid_varying_vox_sizes2)


# Check that the output directory is the same for both runs
assert result2.output_dir == result1.output_dir

# Change the voxel sizes to resample the NIfTI files to for one of the files
mrgrid_varying_vox_sizes2.inputs.voxel[2] = [0.25]

# Result from previous run is reused as the task and inputs are identical
with submitter:
    result3 = submitter(mrgrid_varying_vox_sizes2)

# The output directory will be different as the inputs are now different
assert result3.output_dir != result1.output_dir

Note that for file objects, the contents of the files are used to calculate the hash
not their paths. Therefore, when inputting large files there might be some additional
overhead on the first run (the file hashes themselves are cached by path and mtime so
shouldn't need to be recalculated unless they are modified). However, this makes the
hashes invariant to file-system movement. For example, changing the name of one of the
files in the nifti directory won't invalidate the hash.

In [ ]:
# Rename a NIfTI file within the test directory
first_file = next(nifti_dir.iterdir())
first_file.rename(first_file.with_name("first.nii.gz"))

mrgrid_varying_vox_sizes3 = MrGrid().split(
    ("input", "voxel"),
    input=nifti_dir.iterdir(),
    voxel=VOXEL_SIZES
)

# Result from previous run is reused as the task and inputs are identical
with submitter:
    result4 = submitter(mrgrid_varying_vox_sizes2)

# Check that the output directory is the same for both runs
assert result4.output_dir == result1.output_dir

See [Caches and hashes](../explanation/hashing-caching.html) for more details on how inputs
are hashed for caching and issues to consider.

## Environments

Work in progress...

See [Containers and Environments](../explanation/environments.rst) for more details.

## Provenance and auditing

Work in progress...